<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_03_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

url = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/Antibiotic_70K_patinets.csv'
data = pd.read_csv(url)

In [2]:
# [1]

data.describe()

,ID,total_d,high_d,low_d,none_high_d,high_p,cdiff,age_5
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.00000
mean,35000.500000,8.049771,3.774386,6.131871,4.275386,42.254031,0.021471,2.74870
std,20207.403759,13.885638,8.795789,11.502711,7.950063,39.836986,0.144951,1.56102
min,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000
25%,17500.750000,2.000000,0.000000,2.000000,1.000000,0.000000,0.000000,1.00000
50%,35000.500000,4.000000,2.000000,3.000000,2.000000,37.500000,0.000000,3.00000
75%,52500.250000,8.000000,4.000000,6.000000,4.000000,80.000000,0.000000,4.00000
max,70000.000000,526.000000,368.000000,397.000000,294.000000,100.000000,1.000000,7.00000


In [3]:
# [2]

data2 = data.groupby('age_5').agg({'high_p': 'mean'})
data2 = data2.sort_values(by='high_p', ascending=False)

print(f'Answer [2] : {data2.index[0]}')

Answer [2] : 7


In [4]:
# [3]

import math

# number of NAs
no_na = data.isna().sum().sum()

# find mean and standard deviation
mean = math.floor(data['total_d'].mean() * 100) / 100
std = math.floor(data['total_d'].std() * 100) / 100

# define upper limit and lower limit
ul = mean + 3 * std
ll = mean - 3 * std

# tag outlier
data3 = data.assign(outlier=data.apply(lambda x:
                                       1 if (x['total_d'] > ul) | (x['total_d'] < ll)
                                       else 0, axis=1))

# count outlier patient
cnt = data3[data3['outlier'] == 1]['ID'].count()

print(f'Answer [3] : {no_na}, {cnt}')

Answer [3] : 0, 1258


In [5]:
# [4] corr

correl = data[['age_5', 'high_p']].corr(method='pearson')

print(f'Answer [4] : {math.floor(correl.iloc[0][1] * 100) / 100}')

Answer [4] : 0.12


In [6]:
# [5] ttest_ind

from scipy import stats
import math

cdiff_1 = data[data['cdiff'] == 1]
cdiff_0 = data[data['cdiff'] == 0]

# find difference between two means
mean_diff = math.floor(
            abs(cdiff_1['high_p'].mean() - cdiff_0['high_p'].mean()) *100) / 100

# perform ttest_ind
t_val, p_val = stats.ttest_ind(cdiff_1['high_p'], cdiff_0['high_p'])

# decision: reject H0 or not
if p_val < 0.05:
    reject = True
else:
    reject = False

print(f'Answer [5] : {mean_diff}, Mean is not same = {reject}')

Answer [5] : 4.79, Mean is not same = True


In [7]:
# [6] LogisticRegression, array - get index with certain value or condition,
#     enumerate, list comprehension, array - join array items into one string

from sklearn.linear_model import LogisticRegression

# define X variables
X_cols = list(data.columns[1:4]) + list(data.columns[5:6]) + list(data.columns[7:8])

# make training set
train_X = data[X_cols]
train_y = data['cdiff']

# train
lr = LogisticRegression(C=100000, random_state=1234, penalty='l2', solver='newton-cg')
model = lr.fit(train_X, train_y)

# find index of coef_ with minus value
idx = [i for i, coeff in enumerate(model.coef_[0]) if coeff < 0]

# find variable names with the found index
variables = ', '.join([X_cols[i] for i in idx])

print(f'Answer [6] : {variables}')

Answer [6] : high_d, low_d
